In [89]:
import tensorflow as tf

model = tf.keras.models.load_model('poster1.h5')

In [90]:
import os
c = os.listdir("data2")
#c.remove(".DS_Store") #['.DS_Store', '류승룡', '오달수']
#c.remove("poster") #포스터는 실제 예측데이터로 쓸거야!
print(c)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']


In [91]:
import cv2
from matplotlib import pyplot as plt

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

img = cv2.imread('./poster/test1.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3,3)
print(faces)

[[403 215 142 142]
 [247 427  76  76]
 [723 324  95  95]
 [866 346 102 102]
 [ 18 370 106 106]]


In [ ]:
import os
def detect(detect_img):
    detect_img = np.asarray(detect_img)/255.0
    a = detect_img.reshape(1,-1,60,3)
    label = " "
    pred = model.predict(a)
    max = np.max(pred)
    idx = np.argmax(pred)
    label = c[idx]
    return label, max

In [92]:
from matplotlib import font_manager, rc
import platform 

if platform.system() == 'Windows': 
    font_path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname = font_path).get_name()
    rc('font', family = font_name)
elif platform.system() == 'Darwin':
    font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'
    rc('font', family='AppleGothic')
else: 
    print('Check your OS system')

In [93]:
from PIL import ImageFont, ImageDraw,Image

In [94]:
import unicodedata

In [95]:
labels = []

In [96]:
result = {'a' : '김향기', 'b' : '류승룡', 'c' : '마동석', 'd' : '설경구', 'e' : '송강호', 'f' : '오달수', 'g' : '유해진', 'h' : '이정재',
         'g' : '전지현', 'h' : '정진영', 'i' : '하정우', 'j' : '황정민'}

In [97]:
from PIL import ImageFont, ImageDraw,Image
detect_img = []
white_color = (255, 255, 255)

img_list = []
pred_max_list = []

for (x,y,w,h) in faces:
    
    cropped_c = img[y : y + h, x : x + w] # 사각형 없는 이미지로 따서 img_list 에 추가
    
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0),2)
    
   # cropped = img[y - int(h/4):y + h + int(h/4), x - int(w/4):x + w + int(w/4)]
    
    cropped = img[y : y + h, x : x + w]
    cropped = cv2.cvtColor(cropped, cv2.COLOR_RGB2BGR)
    
    cropped = cv2.resize(cropped, (60, 60))

    cropped.reshape(-1, 60, 60, 3)
    
    #cropped = cv2.resize(cropped,(60, 60))
    
    
    # 한글 Text 설정을 위한 Pillow 
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    
    # 한글 폰트 설정
    font = ImageFont.truetype(font_path, 15)
    
    # 레이블 인덱스, 최대 값 가져오기
    label, pred_max = detect(cropped)
    labels.append(label)
    pred_max_list.append(pred_max)

    if (pred_max >= 0.85):
        name = result[label]
    else:
        name = "Unknown"
        
    # ㅎ ㅏ ㅈ ㅓ ㅇ ㅇ ㅜ 로 나오는 한글 깨짐 설정
    uni1 = unicodedata.normalize('NFD',name)
    uni2 = unicodedata.normalize('NFC',uni1)
        
    # 이미지에 배우 이름 표시
    draw.text((x,y), uni2, font = font, fill = (255,255,255))
    img = np.array(img)

    detect_img.append(cropped)
    #print(detect(cropped))
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    
    cropped_c = cv2.cvtColor(cropped_c, cv2.COLOR_BGR2RGB)
    img_list.append(cropped_c)
    

cv2.imshow('Image view', img)저장
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

a
uni1 :  Unknown
uni2 :  Unknown
a
uni1 :  Unknown
uni2 :  Unknown
g
uni1 :  전지현
uni2 :  전지현
c
uni1 :  마동석
uni2 :  마동석
j
uni1 :  Unknown
uni2 :  Unknown


-1

In [ ]:
for key in result.keys():
    print(key)

### 출연자 명단 출력

In [ ]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font)


poster_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # 포스터 이미지 원본
plt.imshow(poster_img)
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
plt.show()


print("\n********************** 출연자 명단 **********************")

num = 0
plot_num = 1
for test_img_t in img_list: # 포스터 얼굴
    
    
    if (pred_max_list[num] >= 0.85):
        name = labels[num]
        res = result[name]
    else:
        res = "Unknown"
    
    w_num = (int(len(img_list) / 3) + 1)
    
    plt.subplot(w_num, 3, plot_num)
    plt.title(res)
    plt.imshow(test_img_t)
    
    # 축 눈금 감추기
    ax = plt.gca()
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    
    num += 1
    plot_num += 1

plt.tight_layout()
plt.show()

In [ ]:
a